In [1]:
import cmath
from copy import copy, deepcopy
import itertools

from complex import *
from ComplexVector import *
from FuncClasses import *
from RegionClasses import *
from regions import *
from utils import *

# import cvxpy as cp
import numpy as np
import sympy as sym
from z3 import *
# import scipy as sp

# z3

In [2]:
# No difference if both Complex or both Real
a = ComplexVector('a', 10)
c = Complex('c')

coeffs = [c]
for i in range(10):
    coeffs.append(a[i])
    
def barrier(z):
    B_vec = [
        1,
        z[0]**2,
        z[1]**2,
        np.conj(z[0])**2,
        np.conj(z[1])**2,
        z[0]*z[1],
        z[0]*np.conj(z[0]),
        z[0]*np.conj(z[1]),
        z[1]*np.conj(z[0]),
        z[1]*np.conj(z[1]),
        np.conj(z[0])*np.conj(z[1])
    ]
    B = 0
    for cf, term in zip(coeffs, B_vec):
        B += to_complex(cf)*ComplexExpr(term.real, term.imag)
    return B

def barrier_z3(z):
    B_vec = [
        1,
        z[0]**2,
        z[1]**2,
        z[0].conj()**2,
        z[1].conj()**2,
        z[0]*z[1],
        z[0]*z[0].conj(),
        z[0]*z[1].conj(),
        z[1]*z[0].conj(),
        z[1]*z[1].conj(),
        z[0].conj()*z[1].conj()
    ]
    B = 0
    for cf, term in zip(coeffs, B_vec):
        B += to_complex(cf)*term
    return B

A = np.matrix(
    [[2,0,0,0,1,0,0,0,0,0],
     [0,0,-2,0,0,0,0,0,0,-1],
     [0,2,0,0,-1,0,0,0,0,0],
     [0,0,0,0,0,1,-1,-1,-1,0],
     [0,0,0,0,0,0,1,-1,0,0],
     [0,0,0,-2,0,0,0,0,0,1],
    ])

conj_a = [
    a[2].conj(),
    a[3].conj(),
    a[0].conj(),
    a[1].conj(),
    a[9].conj(),
    a[5].conj(),
    a[7].conj(),
    a[6].conj(),
    a[8].conj(),
    a[4].conj(),
]



conj_exps = []
# conj_exps = [c == c.conj()]
for l, r in zip(a, conj_a):
    conj_exps.append(simplify(l == r))

exps = []
lhs_vec = np.dot(A, a)
for i in range(6):
    exps.append(simplify(lhs_vec.item((0,i)) == 0))
    
s = z3.Solver()
# Numerical matrix from differential
s.add(exps)

# Coefficients equal up to complex conjugate
s.add(conj_exps)

# Not all coefficients are 0 - makes coefficients very complicated
# s.add(Not(And([e == 0 for e in a] + [c == 0])))

# Barrier Function at initial and unsafe points
# Select sampels
s.add(barrier([1,0]).r <= 0, barrier([1,0]).i == 0)
s.add(barrier([0,1]).r > 0, barrier([0,1]).i == 0)
for k in range(4):
    s.add(barrier([np.sqrt(0.9), np.exp(1j*k*np.pi/2)*np.sqrt(0.1)]).r <= 0, barrier([np.sqrt(0.9), np.exp(1j*k*np.pi/2)*np.sqrt(0.1)]).i == 0)
    s.add(barrier([np.sqrt(0.1), np.exp(1j*k*np.pi/2)*np.sqrt(0.9)]).r > 0, barrier([np.sqrt(0.1), np.exp(1j*k*np.pi/2)*np.sqrt(0.9)]).i == 0)

# Random samples
# for k in range(10):
#     # Init sample
#     r = np.random.uniform(np.sqrt(0.9), 1)
#     ang0 = np.random.uniform(0, 2*np.pi)
#     ang1 = np.random.uniform(0, 2*np.pi)
#     z0 = r * np.exp(1j*ang0)
#     z1 = np.sqrt(1 - r**2) * np.exp(1j*ang1)
#     Z = [z0, z1]
#     s.add(barrier(Z).r <= 0, barrier(Z).i == 0)
#     # Unsafe sample
#     r = np.random.uniform(0, np.sqrt(0.1))
#     ang0 = np.random.uniform(0, 2*np.pi)
#     ang1 = np.random.uniform(0, 2*np.pi)
#     z0 = r * np.exp(1j*ang0)
#     z1 = np.sqrt(1 - r**2) * np.exp(1j*ang1)
#     Z = [z0, z1]
#     s.add(barrier(Z).r > 0, barrier(Z).i == 0)

# ForAll Expressions
# def ComplexForAll(ls, phi):
#     return ForAll([qv for v in ls for qv in [v.r, v.i]], phi)
# z = ComplexVector('z', 2)
# init = Implies(And(z[0].len_sqr() + z[1].len_sqr() == 1, z[0].len_sqr() >= 0.9), And(barrier_z3(z).r <= 0, barrier_z3(z).i == 0))
# unsafe = Implies(And(z[0].len_sqr() + z[1].len_sqr() == 1, z[0].len_sqr() < 0.1), And(barrier_z3(z).r > 0, barrier_z3(z).i == 0))
# s.add(ComplexForAll(z, init))
# s.add(ComplexForAll(z, unsafe))

models = print_all_models(s)

sat
Model 1
[a__6.i = 0,
 a__1.i = 0,
 a__9.r = 0,
 a__5.i = 0,
 a__0.i = 0,
 a__1.r = 0,
 c.i = 0,
 a__9.i = 0,
 a__2.i = 0,
 a__2.r = 0,
 a__4.r = 0,
 a__0.r = 0,
 a__8.i = 0,
 a__4.i = 0,
 a__7.i = 0,
 a__3.i = 0,
 a__3.r = 0,
 a__7.r = -2500000000000000/1999999999999999,
 c.r = 5999999999999999/3999999999999998,
 a__8.r = 0,
 a__6.r = -2500000000000000/1999999999999999,
 a__5.r = -5000000000000000/1999999999999999]
unsat


In [3]:
z_sym = [sym.Symbol('z0', complex=True), sym.Symbol('z1', complex=True)]
B = sym.Function('B')
B_vars = [
        1,
        z_sym[0]**2,
        z_sym[1]**2,
        np.conj(z_sym[0])**2,
        np.conj(z_sym[1])**2,
        z_sym[0]*z_sym[1],
        z_sym[0]*np.conj(z_sym[0]),
        z_sym[0]*np.conj(z_sym[1]),
        z_sym[1]*np.conj(z_sym[0]),
        z_sym[1]*np.conj(z_sym[1]),
        np.conj(z_sym[0])*np.conj(z_sym[1])
    ]

def get_barrier(model):
    b = 0
    for c, v in zip(coeffs, B_vars):
        r = round(float(model[c.r].numerator_as_long())/float(model[c.r].denominator_as_long()), 2)
        i = round(float(model[c.i].numerator_as_long())/float(model[c.i].denominator_as_long()), 2)
        co = r + 1j*i
        b += co * v
    return b

In [4]:
b = get_barrier(models[0])*4/5
eq = sym.Eq(B(z_sym[0], z_sym[1]), b)
eq

Eq(B(z0, z1), -2.0*z0*conjugate(z0) - 1.0*z0*conjugate(z1) - 1.0*z1*conjugate(z0) + 1.2)

$c = 3/2,
 a__0 = 0,
 a__1 = 0,
 a__2 = 0,
 a__3 = 0,
 a__4 = 0,
 a__5 = -5/2,
 a__6 = -5/4,
 a__7 = -5/4,
 a__8 = 0,
 a__9 = 0$

Given a function ($\dot{x} = f(x)$) and a the complex system of size $n$

Want to compute a $k$-degree barrier function: $B^{(k)}(z) = b^{(k)}(z, \overline{z})$ such that $b(z,u) = \sum_{R_{sum} + S_{sum} \leq k} z^{R} u^{S}$

Guess that $k \leq 2^n$ should be good for most cases

1. Put coefficients and terms into an array Barr = list((coeffs, terms))
2. Calculate dBdz from Barr, modifying terms and coeffs as needed
3. Calculate dBdzconj from Barr, "
4. Calculate dBdt from dBdz, dBdzconj, f and fbar (also needs to be a list of coefficients and terms)
5. Generate A from individual coeff in dBdt, then create constraints (sum(row(A)) == 0)
6. Calculate Barrconj from Barr to get other constraints
7. Sample data from $Z_\text{init}$, $Z_\text{unsafe}$ into Barr to calculate constant a_0

In [5]:
# f = lambda a,b: np.array([0 - I * (a + b)/Sqrt(2), 0 - I * (a - b)/Sqrt(2)])
# fbar = lambda a,b: np.array([I * (a.conj() + b.conj())/Sqrt(2), I * (a.conj() - b.conj())/Sqrt(2)])

k = 2
n = 2

terms = generate_term_powers(k, n)
c0 = Complex('c')
coeffs = [c] + ComplexVector('a', len(terms) - 1)

z = 0-I/np.sqrt(2)
z = 0-I
f = FuncVec([FuncSum([FuncTerm(z, (1,0,0,0)),FuncTerm(z, (0,1,0,0))]),
             FuncSum([FuncTerm(z, (1,0,0,0)),FuncTerm(0-z, (0,1,0,0))])
            ])
f

FuncVec([FuncSum([FuncTerm([0 - 0 + (0 - 1)*I, (1, 0, 0, 0)]), FuncTerm([0 - 0 + (0 - 1)*I, (0, 1, 0, 0)])]), FuncSum([FuncTerm([0 - 0 + (0 - 1)*I, (1, 0, 0, 0)]), FuncTerm([0 - (0 - 0) + (0 - (0 - 1))*I, (0, 1, 0, 0)])])])

In [6]:
barrier = FuncSum(list([FuncTerm(c, t) for c, t in zip(coeffs, terms)]))
dbdz = FuncVec([diff(barrier, i) for i in range(n)])
dbdzconj = FuncVec([diff(barrier, i) for i in range(n, 2*n)])
dbdt = (dbdz * f) + (dbdzconj * f.conj())
barr_minus_conj = barrier - barrier.conj()
barrier

FuncSum([FuncTerm([c.r + (c.i)*I, [0, 0, 0, 0]]), FuncTerm([a__0.r + (a__0.i)*I, [0, 0, 0, 1]]), FuncTerm([a__1.r + (a__1.i)*I, [0, 0, 0, 2]]), FuncTerm([a__2.r + (a__2.i)*I, [0, 0, 1, 0]]), FuncTerm([a__3.r + (a__3.i)*I, [0, 0, 1, 1]]), FuncTerm([a__4.r + (a__4.i)*I, [0, 0, 2, 0]]), FuncTerm([a__5.r + (a__5.i)*I, [0, 1, 0, 0]]), FuncTerm([a__6.r + (a__6.i)*I, [0, 1, 0, 1]]), FuncTerm([a__7.r + (a__7.i)*I, [0, 1, 1, 0]]), FuncTerm([a__8.r + (a__8.i)*I, [0, 2, 0, 0]]), FuncTerm([a__9.r + (a__9.i)*I, [1, 0, 0, 0]]), FuncTerm([a__10.r + (a__10.i)*I, [1, 0, 0, 1]]), FuncTerm([a__11.r + (a__11.i)*I, [1, 0, 1, 0]]), FuncTerm([a__12.r + (a__12.i)*I, [1, 1, 0, 0]]), FuncTerm([a__13.r + (a__13.i)*I, [2, 0, 0, 0]])])

Want a__11, a__10 and a__7 (possibly a__6)

In [7]:
s = Solver()
s.add([simplify(c == 0) for c in dbdt.get_coeffs()])
s.add([simplify(c == 0) for c in barr_minus_conj.get_coeffs()])
# s.add(Or([Not(a == 0) for a in coeffs]))
init = lambda x: s.add(simplify(barrier.apply(x).r <= 0), simplify(barrier.apply(x).i == 0))
unsafe = lambda x: s.add(simplify(barrier.apply(x).r > 0), simplify(barrier.apply(x).i == 0))
init([1,0])
unsafe([0,1])
for k in range(4):
    init([np.sqrt(0.9), np.exp(1j*k*np.pi/2)*np.sqrt(0.1)])
    unsafe([np.sqrt(0.1), np.exp(1j*k*np.pi/2)*np.sqrt(0.9)])

# s.add(c.r < 10, -10 < c.r)
# Restricts to our normal barrier function
s.add(c == 0)

ms = print_all_models(s)

sat
Model 1
[a__6.i = 0,
 a__0.i = 0,
 c.i = 0,
 a__2.r = 0,
 a__4.i = 0,
 a__7.i = 0,
 a__10.i = 0,
 a__3.r = 0,
 a__13.r = 0,
 a__5.r = 0,
 a__6.r = 749999999999999992727286219571085000000000000000000000000000000000000000000000000000000000000000000/499999999999999960788456797856571089012777484922496724882775317387071664898870725753599284127762469,
 a__8.r = 0,
 a__1.i = 0,
 a__9.r = 0,
 a__5.i = 0,
 a__1.r = 0,
 a__9.i = 0,
 a__12.r = 0,
 a__2.i = 0,
 a__11.i = 0,
 a__4.r = 0,
 a__0.r = 0,
 a__12.i = 0,
 a__8.i = 0,
 a__10.r = -625000000000000003162715025133678187349864163661040005223927125080625000000000000000000000000000000/499999999999999960788456797856571089012777484922496724882775317387071664898870725753599284127762469,
 a__3.i = 0,
 a__11.r = -500000000000000013598143830696271374699728327322080010447854250161250000000000000000000000000000000/499999999999999960788456797856571089012777484922496724882775317387071664898870725753599284127762469,
 a__13.i = 0,
 a__7.r = -625000000000

In [8]:
barrier.get_sym_sum(ms[0])

-1.0*z0*conjugate(z0) - 1.25*z0*conjugate(z1) - 1.25*z1*conjugate(z0) + 1.5*z1*conjugate(z1)

In [9]:
simp_b = - z_sym[0]**2 + 2 * z_sym[0] * z_sym[1] - 2 * z_sym[0] * np.conj(z_sym[1]) + z_sym[1] ** 2 - 2 * np.conj(z_sym[0]) * z_sym[1] - np.conj(z_sym[0]) ** 2 + 2 * np.conj(z_sym[0]) * np.conj(z_sym[1]) + np.conj(z_sym[1])**2
sym.sympify(simp_b)

-z0**2 + 2*z0*z1 - 2*z0*conjugate(z1) + z1**2 - 2*z1*conjugate(z0) - conjugate(z0)**2 + 2*conjugate(z0)*conjugate(z1) + conjugate(z1)**2

## Other Hamiltonians

In [10]:
def find_k_barrier_sample(k, H, init, unsafe, restrict_c = False, prec=2, samples=50):
    z = 0-I
    n = round(len(H))
    sums = []
    for i in range(n):
        terms = []
        for j in range(n):
            t = [0]*(2*n)
            t[j] = 1
            t = tuple(t)
            terms.append(FuncTerm(z * H[i][j], t))
        sums.append(FuncSum(terms))
    f_vec = FuncVec(sums)
    print("Dynamical system converted...")
    
    term_powers = generate_term_powers(k, n)
    c0 = Complex('c')
    coeffs = [c0] + ComplexVector('a', len(term_powers) - 1)
    barrier = FuncSum(list([FuncTerm(c, t) for c, t in zip(coeffs, term_powers)]))
    print("Barrier made...")
    dbdz = FuncVec([diff(barrier, i) for i in range(n)])
    dbdzconj = FuncVec([diff(barrier, i) for i in range(n, 2*n)])
    dbdt = (dbdz * f_vec) + (dbdzconj * f_vec.conj())
    print("Differential of barrier made...")
    
    barr_minus_conj = barrier - barrier.conj()
    
    s = Solver()
    s.add([simplify(coeff == 0) for coeff in dbdt.get_coeffs()])
    s.add([simplify(coeff == 0) for coeff in barr_minus_conj.get_coeffs()])
    # s.add(Or([Not(a == 0) for a in coeffs]))
    
    # Samples
    init.add_conditions(s, barrier, n=samples)
    unsafe.add_conditions(s, barrier, n=samples)

    idx = []
    for i in range(n):
        temp = [0]*n
        temp[i] = 1
        idx.append(term_powers.index(temp*2))
    prob_coeffs = [coeffs[i] for i in idx]
    s.add(sum(prob_coeffs) == 1)
    
    # s.add(c.r < 10, -10 < c.r)
    # Restricts to our normal barrier function
    if restrict_c: s.add(c == 0)
    print("Solver ready...")
    ms = print_all_models(s)
    return barrier.get_sym_sum(ms[0], prec)

In [11]:
H = [[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]]
# H = [[1,1],[1,-1]]
init = InitRegion(near_0)
unsafe = UnsafeRegion(near_1)
b = find_k_barrier_sample(2, H, init, unsafe, restrict_c = True)
b

Dynamical system converted...
Barrier made...
Differential of barrier made...
Solver ready...
sat
Model 1
[a__6.i = 0,
 a__0.i = 0,
 c.i = 0,
 a__2.r = 0,
 a__4.i = -82627578260632756899755259256208767026801641140795926299062378690956290169241145765787705604162020279/197105481961735871600205117549431531094949830969468967637992434238473219855151190568812008066211366180,
 a__7.i = 0,
 a__10.i = 0,
 a__3.r = 24532872275508009234100408875781172815205432793597728037653105883851815366821544774685558188195686508/49276370490433967900051279387357882773737457742367241909498108559618304963787797642203002016552841545,
 a__13.r = -12266436137754004617050204437890586407602716396798864018826552941925907683410772387342779094097843254/49276370490433967900051279387357882773737457742367241909498108559618304963787797642203002016552841545,
 a__5.r = 0,
 a__6.r = 31984145648155162084542677954873198690055480026953898520271959224858670705360931456642845379814012820/98552740980867935800102558774715765547474915

z0**2*(-0.25 + 0.42*I) + z0*z1*(0.5 - 0.84*I) - 2.25*z0*conjugate(z0) - 2.75*z0*conjugate(z1) + z1**2*(0.25 - 0.42*I) - 2.75*z1*conjugate(z0) + 3.25*z1*conjugate(z1) + (-0.25 - 0.42*I)*conjugate(z0)**2 + (0.5 + 0.84*I)*conjugate(z0)*conjugate(z1) + (0.25 + 0.42*I)*conjugate(z1)**2

In [12]:
# Setup for an X-gate - should fail
H = [[.5,-.5],[-.5,.5]]
init = InitRegion(near_0)
unsafe = UnsafeRegion(near_1)
# old_find_k_barrier(2, H, init, unsafe, restrict_c = False)

In [13]:
H = [[.5,-.5],[-.5,.5]]
init = InitRegion(near_unphased)
unsafe = UnsafeRegion(very_phased)
find_k_barrier_sample(2, H, init, unsafe, restrict_c = False)

Dynamical system converted...
Barrier made...
Differential of barrier made...
Solver ready...
sat
Model 1
[a__6.i = 0,
 a__0.i = -5408437740078702515621667662855418173644193542713983178864416675990305149634567594076759/299529243789137243027689327820677992291661303360327794324835891616529708000000000000000000,
 c.i = 0,
 a__2.r = 90696098514467821888628582918745018209092372796546457935310933014765936293956699456493939/99843081263045747675896442606892664097220434453442598108278630538843236000000000000000000,
 a__4.i = 0,
 a__7.i = 0,
 a__10.i = 0,
 a__3.r = 0,
 a__13.r = 0,
 a__5.r = 90696098514467821888628582918745018209092372796546457935310933014765936293956699456493939/99843081263045747675896442606892664097220434453442598108278630538843236000000000000000000,
 a__6.r = 1/2,
 a__8.r = 0,
 a__1.i = 0,
 a__9.r = 90696098514467821888628582918745018209092372796546457935310933014765936293956699456493939/99843081263045747675896442606892664097220434453442598108278630538843236000000000000000000

0.5*z0*conjugate(z0) - 17.72*z0*conjugate(z1) + z0*(0.91 + 0.02*I) - 17.72*z1*conjugate(z0) + 0.5*z1*conjugate(z1) + z1*(0.91 + 0.02*I) + (0.91 - 0.02*I)*conjugate(z0) + (0.91 - 0.02*I)*conjugate(z1) - 0.86

In [14]:
# TODO: Try out new Hamiltonians and regions
H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
# init = InitRegion(ctrl_near_0)
# init = InitRegion(tgt_near_0)
init = InitRegion(near_00)
unsafe = UnsafeRegion(near_11)
find_k_barrier_sample(2, H, init, unsafe, restrict_c = False)

# Try get working
# init = InitRegion(near_10)
# unsafe = UnsafeRegion(away_from_11_10)

Dynamical system converted...
Barrier made...
Differential of barrier made...
Solver ready...
sat
Model 1
[a__6.i = 0,
 a__31.i = 0,
 a__23.i = 0,
 a__20.i = 0,
 a__28.i = 0,
 a__37.i = 0,
 a__15.r = 0,
 a__32.i = 0,
 a__0.i = 0,
 a__2.r = 0,
 a__43.i = 0,
 a__30.i = 0,
 a__38.r = 0,
 a__30.r = 511188991993845807071276621070955265051648417775135042301736725057608/412174788782400283827437589505623458226509175576544588533543234764281,
 a__31.r = 0,
 a__37.r = 0,
 a__17.r = 0,
 a__25.r = 0,
 a__7.i = 0,
 a__10.i = 0,
 a__3.r = 0,
 a__39.r = -99014203211445523243839031565331806825139242198590453768193490293327/412174788782400283827437589505623458226509175576544588533543234764281,
 a__13.r = 0,
 a__8.r = 0,
 a__33.r = 0,
 a__19.i = 0,
 a__6.r = 0,
 a__34.i = 0,
 a__38.i = 0,
 a__1.r = 0,
 a__36.i = 0,
 a__26.i = 0,
 a__16.i = 0,
 a__2.i = 0,
 a__16.r = 0,
 a__11.i = 0,
 a__19.r = 0,
 a__0.r = 0,
 a__29.i = 0,
 a__10.r = 0,
 a__8.i = 0,
 a__12.i = 0,
 a__11.r = 0,
 a__22.i = 0,
 a__14.r = 0,

-0.24*z0*conjugate(z0) + 1.24*z1*conjugate(z1) + 0.09

# Notes

So far have developed a function that is able to generate a $k$-barrier given a system and some integer $k$.

The barriers generated need to be checked using the solver constraints generated previously (akin to 4.5).

The generation of barriers using eigenvalues needs to be checked.

After that, begin investigating perturbed systems (Chapter 7 of Intro to Quantum Mechanics).